In [65]:
!pip install ipython-autotime
%load_ext autotime

time: 175 µs (started: 2021-03-15 12:11:22 +00:00)


In [1]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 4.4MB/s 


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs=SentimentIntensityAnalyzer()

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/bhukyavamshirathod/Sentimental-Analysis/main/Train.csv')
df.head()

In [ ]:
import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
len(stopword_list)

In [ ]:
# removing html tag
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text

In [ ]:
# expand contractions
!pip install contractions
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand

In [ ]:
# removal of special character
import re
def remove_sp(text):
  pattern=r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text

In [ ]:
# remove stop words
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

def remove_stop_words(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text

In [ ]:
# 1: lower case
# 2: html tag
# 3: contraction
# 4: tokenize
# 5: stop words
df.tweet=df.tweet.apply(lambda x:x.lower())
df.tweet=df.tweet.apply(html_tag)
df.tweet=df.tweet.apply(con)
df.tweet=df.tweet.apply(remove_sp)
df.tweet=df.tweet.apply(remove_stop_words)

df.head()

In [ ]:
# df['compound']=df['news_article'].apply(lambda x: vs.polarity_scores(x)['compound'])
df['compound']=df['tweet'].apply(lambda x: vs.polarity_scores(x)['compound'])

In [ ]:
conditions = [
    (df['compound'] >= 0.05),
    (df['compound'] > -0.05) & (df['compound'] < 0.05),
    (df['compound'] <= -0.05)
    ]

# create a list of the values we want to assign for each condition
values = ['Positive', 'Neutral', 'Negative']

# create a new column and use np.select to assign values to it using our lists as arguments
df['tier'] = np.select(conditions, values)

# display updated DataFrame
df

In [ ]:
df.head()

In [ ]:
x = df.iloc[:,2].values
y = df.iloc[:,-1].values

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 0,stratify = y)
x_train.shape

In [ ]:
np.unique(y_train,return_counts=True)

In [ ]:
np.unique(y_test,return_counts=True)

In [ ]:
# Pipeline = Combine above 2 steps
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
# Syntax : Pipeline([('Variable 1',Method 1()),('Variable 2',Method 2())])

In [ ]:
text_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())])

In [ ]:
text_model.fit(x_train,y_train)

In [ ]:
y_pred = text_model.predict(x_test)
y_pred

In [ ]:
np.unique(y_pred,return_counts=True)

In [ ]:
y_test

In [ ]:
# Evaluation
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
accuracy_score(y_pred,y_test)*100

In [ ]:
confusion_matrix(y_pred,y_test)

In [ ]:
print(classification_report(y_pred,y_test))

In [ ]:
text_model.predict(["#people aren't protesting #trump because a #republican not every on e won never hw is dumbo kjkjwe @#$%^&*Â€Â¦ "])

In [61]:
!pip install streamlit
!pip install pyngrok==4.1.1
from pyngrok import ngrok

In [64]:
%%writefile app.py
import streamlit as st
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

# remove html tag
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text

# !pip install contractions
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand

# removal of special character
import re
def remove_sp(text):
  pattern=r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text


# remove stop words
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

def remove_stop_words(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text


st.title("Tweet's Sentiment Analysis")

df = pd.read_csv('https://raw.githubusercontent.com/bhukyavamshirathod/Sentimental-Analysis/main/Train.csv')

df.tweet=df.tweet.apply(lambda x:x.lower())
df.tweet=df.tweet.apply(html_tag)
df.tweet=df.tweet.apply(con)
df.tweet=df.tweet.apply(remove_sp)
df.tweet=df.tweet.apply(remove_stop_words)

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs=SentimentIntensityAnalyzer()

df['compound']=df['tweet'].apply(lambda x: vs.polarity_scores(x)['compound'])

conditions = [
    (df['compound'] >= 0.05),
    (df['compound'] > -0.05) & (df['compound'] < 0.05),
    (df['compound'] <= -0.05)
    ]

# create a list of the values we want to assign for each condition
values = ['Positive', 'Neutral', 'Negative']

# create a new column and use np.select to assign values to it using our lists as arguments
df['tier'] = np.select(conditions, values)

x = df.iloc[:,2].values
y = df.iloc[:,-1].values

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
text_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())]) 
text_model.fit(x,y)
select = st.text_input('Enter your message')

if(st.markdown(
    '<span class="badge badge-pill badge-success"> Badge </span>',
    unsafe_allow_html=True
)):
  op = text_model.predict([select])
  ans=op[0]

  if ans == 'Positive':
    st.success("Positive 🙂")
  if ans == 'Negative':
    st.error("Negative 😠")
  if ans== 'Neutral':
    st.warning("Neutral 😐")

Overwriting app.py


In [63]:
!nohup streamlit run app.py &
url = ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://4d3dadbdd5ac.ngrok.io'